In [1]:
using Distributions, TimeIt, ProgressMeter, PyPlot, CSV
include("zz_structures_DuLuSuSe.jl")
include("mbsampler.jl")

get_clustering (generic function with 1 method)

In [2]:
m = CSV.read("pd_speech_features.csv", header=false);

In [3]:
X = zeros(size(m,1)-2,size(m,2)-2)
for i in 3:size(m,1) 
    for j in 2:size(m,2)-1 
        X[i-2,j-1] = parse(m[i,j])
    end
end
mu, sd = vec(mean(X[:,2:end],1)), vec(std(X[:,2:end],1))
X_normalised = zeros(size(X))
for i in 1:size(X,1) 
    X_normalised[i,2:end] = (X[i,2:end]-mu)./sd
end
X_normalised = X_normalised';
y = vec([parse(m[i,end]) for i in 3:size(m,1)]);

In [4]:
d, Nobs = size(X_normalised)
σ02 = 1;

In [6]:
prior = SS_prior(d, σ02)
my_ll = ll_logistic(X_normalised,y)
my_model = model(my_ll, prior);

In [ ]:
ξ0 = zeros(d)
epsilon = 3e-5
L = 20
T = 100

xi_samples, hyper_samples, acc = GibbsHMC(my_model, ξ0, epsilon, L, T);

In [ ]:
for i in 1:size(xi_samples,1) 
    plot(acf(xi_samples[i,:], 500))
end
grid(true)